In [1]:
#In this code I'm going to see if we can train a NN from one game and have it predict positions from the next game
#Importing the libraries
from data_loader import FootballDataLoader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, LSTM, GRU
import pickle
import os
import matplotlib.pyplot as plt
from ML_Functions_LoadArrays import *
import numpy as np
import shap
from tensorflow.keras.layers import Masking
from tensorflow.keras.optimizers import Adam
import seaborn as sns
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Masking, GRU, Dropout, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import contextlib
import sys
import os


2025-06-04 12:05:20.596702: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 12:05:20.597149: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 12:05:20.599212: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 12:05:20.604518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749031520.613515  319909 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749031520.61

In [ ]:
def get_xy_from_XG(XGPos1, XGPos2, team):
    if team == 'AAB':
        x = np.array(XGPos1, dtype=float) / 100 * 105 - 52.5  
        y = np.array(XGPos2, dtype=float) / 100 * 70 - 35
    elif team in ['AGF', 'BIF', 'FCK', 'FCN',  'SIF', 'SJE', 'VFF']:
        x = np.array(XGPos1, dtype=float) / 100 * 105 - 52.5  
        y = np.array(XGPos2, dtype=float) / 100 * 68 - 34
    elif team == 'FCM':
        x = np.array(XGPos1, dtype=float) / 100 * 104 - 52  
        y = np.array(XGPos2, dtype=float) / 100 * 65 - 32.5
    elif team in ['LYN', 'RFC']:
        x = np.array(XGPos1, dtype=float) / 100 * 102 - 51  
        y = np.array(XGPos2, dtype=float) / 100 * 65 - 32.5
    elif team == 'VB':
        x = np.array(XGPos1, dtype=float) / 100 * 108 - 54
        y = np.array(XGPos2, dtype=float) / 100 * 68 - 34
    return x, y

def get_shot_time(times_x, t, Us, Ball, XGNumbers, XGPos1, XGPos2, i, verbose=False):
    passed = False
    idxs = np.where(np.isclose(times_x, t, atol=0.03)) #I believe when XGDat says 50 seconds, it means the shot took place between 49-50 seconds, research shows this is usually true, but sometimes the time in XGData is 3 seconds off...
    if len(idxs[0]) == 0:
        if verbose:
            print(f"Warning: No close time found for {t} in times_x.")
        return 0, passed
    
    idxes = idxs[0][0]  # Get the first index where the time is close
    frames = np.arange(max(0 , idxes-40), min(len(Us), idxes+15)) #So we take the whole second, and also half a second before and after, so we take from 48.5-50.5
    for idx in frames:
        if Ball[idx][5] == 0:
             frames = np.delete(frames, np.where(frames == idx)) #take only the idx in which the ball is being played

    if len(frames) == 0: 
        if verbose:
            print(f"Ball not being played.")
        return 0, passed
    
    else:
        shooter = int(XGNumbers[i])
        shooter_x = float(XGPos1[i])
        shooter_y = float(XGPos2[i])
        #print(shooter, shooter_x, shooter_y, t/60)
        shooter_pos_in_data = next((j for j, player_data in enumerate(Us[frames[0]]) if int(player_data[4]) == shooter), None)

        if shooter_pos_in_data is None:
            if verbose:
                print(f"Warning: player {shooter} not found in Us[{frames[0]}]")
            return 0, passed

        distances = np.array([np.sqrt((Us[idx][shooter_pos_in_data][0] - Ball[idx][0])**2 + (Us[idx][shooter_pos_in_data][1] - Ball[idx][1])**2) for idx in frames])
        mask = ((np.abs(np.array([Us[idx][shooter_pos_in_data][0] for idx in frames]) - shooter_x) <= 1.5) &
                (np.abs(np.array([Us[idx][shooter_pos_in_data][1] for idx in frames]) - shooter_y) <= 1.5))
        distances = distances[mask]
        frames = frames[mask]

        if len(distances) == 0:
            if verbose:
                print(f"Warning: player {shooter} not found in Us[{frames[0]}] within 1.5m of where the player is supposed to be.")
            return 0, passed
        
        else : 
            if min(distances) > 2:
                if verbose:
                    print("Ball - Shooter distance > 2")
                return 0, passed
            passed = True
            if verbose:
                print(f"Shot time found at frame {frames[np.argmin(distances)]} with distance {min(distances)}")
           
            return frames[np.argmin(distances)], passed
 
def get_random_sequence_by_number(Team, verbose=False):
    broken_NamesSC = ['Game_SJE_BIF_Score_2_2_Day_2024-11-24Z.pkl', 'Game_SJE_VFF_Score_2_2_Day_2024-09-01Z.pkl', 'Game_SJE_FCN_Score_1_4_Day_2024-10-06Z.pkl']
    NamesXG, NamesSC = SortGames('pippo', Team)
    columns = ['XG', 'Time', 'Half','ball_x', 'ball_y', 'ball_z', 'ball_speed']
    for i in range(1, 30):
        columns += [f'us_{i}_x', f'us_{i}_y', f'us_{i}_speed']
        columns += [f'them_{i}_x', f'them_{i}_y', f'them_{i}_speed']
    

    rows = []
    for igame in range(30):
        if NamesSC[igame] not in broken_NamesSC:
            Time, Ball, Us, Them = SecLoad(Team, NamesSC, igame)


            times_x = np.array([t[0] for t in Time])
            XGs = []
            number_of_fake_shots = 100
            for shot in range(number_of_fake_shots):
                idx = np.random.randint(0, len(times_x))
                i,t = Time[idx]
                XG_value, XG_Team = Niels_model(Us[idx], Them[idx], Ball[idx], Team, verbose=verbose)
                
                if XG_value > 0:
                    xg_moment = XG_value
                    
                    if XG_Team  == Team:
                        xg_moment = XG_value
                    else: 
                        xg_moment = -xg_moment  # If the team is not the one we are looking for, we negate the XG value
                        
                    numbers_us = [int(player[4]) for player in Us[idx]]
                    numbers_them = [int(player[4]) for player in Them[idx]]
                    us_ordered = sorted(range(len(numbers_us)), key=lambda k: numbers_us[k])
                    them_ordered = sorted(range(len(numbers_them)), key=lambda k: numbers_them[k])
                    Us[idx] = [Us[idx][j] for j in us_ordered]
                    Them[idx] = [Them[idx][j] for j in them_ordered]
                    frames_definig_play = np.arange(max(0 , idx-125*2), idx) 
                    Ball_values = np.array([Ball[frame][5] for frame in frames_definig_play])
                    find_stop = np.where(Ball_values == 0)[0][-1] if np.any(Ball_values == 0) else None
                    if find_stop is not None:
                        frames_definig_play = frames_definig_play[find_stop:]
                    for time in frames_definig_play : 
                        row = {}
                        row['XG'] = xg_moment
                        row['Time'] = Time[time][0]
                        row['Half'] = 1 # TBD
                        row['ball_x'] =  Ball[time][0] * np.sign(Ball[time][0]) * np.sign(xg_moment)
                        row['ball_y'] =  Ball[time][1]
                        row['ball_z'] =  Ball[time][2]
                        row['ball_speed'] =  Ball[time][3]
                        for j in range(len(Us[time])):
                            row[f'us_{numbers_us[j]}_x'] = Us[time][j][0]*np.sign(Ball[time][0]) * np.sign(xg_moment)
                            row[f'us_{numbers_us[j]}_y'] = Us[time][j][1]
                            row[f'us_{numbers_us[j]}_speed'] = Us[time][j][3]
                        for j in range(len(Them[time])):
                            row[f'them_{j+1}_x'] = Them[time][j][0]*np.sign(Ball[time][0]) * np.sign(xg_moment)
                            row[f'them_{j+1}_y'] = Them[time][j][1]
                            row[f'them_{j+1}_speed'] = Them[time][j][3]
                    
                        rows.append(row)
                    XGs.append(XG_value)
                            

    total_XG_data = pd.DataFrame(rows, columns=columns)
    return total_XG_data, XGs

#dist_l = []
#tXG_all, Xgs = get_sequence_by_number('AGF', verbose=False) #1 team takes +- 1 min,  
#tXG_all = tXG_all.dropna(axis=1, how='all')
#print("Team AAB done")
#
#for team in ['AGF' , 'AAB' , 'BIF', 'FCK', 'FCM', 'FCN', 'RFC', 'SIF', 'SJE', 'VB', 'VFF', 'LYN']: # missing 
#    tXG, d_l = get_sequence_by_number(team) 
#    print(f"Team {team} done")
#    tXG.to_csv(f'XGData_sequencies_{team}.csv', index=False)